# Tunning KNN

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import  confusion_matrix,recall_score,precision_score,accuracy_score
from imblearn.metrics import geometric_mean_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt




def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    g_mean = geometric_mean_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도 {1:4f}, 재현율 {2:4f}, AUC {3:4f},G_MEAN {4:4f}'.format(accuracy , precision , recall,roc,g_mean))
    

def get_model_train_eval(model,ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)



import warnings
warnings.filterwarnings(action='ignore')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  w

In [2]:
df = pd.read_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/Loan_Train.csv')
df.head()

df = df.drop('Id',axis = 1)


from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

data = df


for e in data.columns:
    if data[e].dtype == 'object':
        labelEncoder.fit(list(data[e].values))
        data[e] = labelEncoder.transform(data[e].values)
        
        # Accommodate the data that has been changed
        df = data
        
y = df.Risk_Flag
X = df.drop('Risk_Flag', axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=101)



In [3]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
# import smote_variants as sv
from sklearn.metrics import roc_auc_score, make_scorer

In [13]:
start = time.time()


pipeline = Pipeline(steps= [("SMOTE", SMOTE(random_state = 101)),
                            ("KNN", KNeighborsClassifier())
                            ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,0.75,1],
    "KNN__n_neighbors": list(range(10,101,10))
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model
smote_best_model = gs_pipeline.best_estimator_

y_validation_preds = smote_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5 ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5, total=   3.5s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5, total=   3.8s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5, total=   3.6s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5, total=   3.9s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.5, total=   3.8s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75 ..............
[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75, total=   4.2s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75 ..............
[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75, total=   4.0s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75 ..............
[CV]  KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75, total=   4.6s
[CV] KNN__n_neighbors=10, SMOTE__sampling_strategy=0.75 ..............
[CV

[CV] .. KNN__n_neighbors=40, SMOTE__sampling_strategy=1, total=   7.3s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5, total=   4.0s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5, total=   4.1s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5, total=   4.3s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5, total=   4.0s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=50, SMOTE__sampling_strategy=0.5, total=   5.2s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.75 ..............
[CV]  KNN__n_neighbors=50, SMOTE__sampling_strategy=0.75, total=   5.0s
[CV] KNN__n_neighbors=50, SMOTE__sampling_strategy=0.75 ..............
[CV] 

[CV] .. KNN__n_neighbors=80, SMOTE__sampling_strategy=1, total=   5.4s
[CV] KNN__n_neighbors=80, SMOTE__sampling_strategy=1 .................
[CV] .. KNN__n_neighbors=80, SMOTE__sampling_strategy=1, total=   5.5s
[CV] KNN__n_neighbors=80, SMOTE__sampling_strategy=1 .................
[CV] .. KNN__n_neighbors=80, SMOTE__sampling_strategy=1, total=   5.7s
[CV] KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5, total=   4.4s
[CV] KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5, total=   4.6s
[CV] KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5, total=   4.7s
[CV] KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5 ...............
[CV]  KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5, total=   4.5s
[CV] KNN__n_neighbors=90, SMOTE__sampling_strategy=0.5 ...............
[CV]  

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 12.4min finished


750.02700 sec


0.8552476034782603

In [14]:
roc_auc_score(y_test, y_validation_preds)

0.8552476034782603

In [6]:
scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('C:/Users/koyounghun/Desktop/찌르레기/Loan_data/KNN_SMOTE.csv',index = False)

In [7]:
start = time.time()


pipeline = Pipeline(steps= [("ADASYN", ADASYN(random_state = 101)),
                            ("KNN", KNeighborsClassifier())
                            ])

param_grid = {
    "ADASYN__sampling_strategy": [0.5,0.75,1],
    "KNN__n_neighbors": list(range(10,101,10))
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model
ada_best_model = gs_pipeline_1.best_estimator_

y_validation_preds = ada_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)

scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10 ..............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10, total=   5.2s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s remaining:    0.0s


[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10, total=   5.1s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10 ..............
[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10, total=   4.8s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10 ..............
[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10, total=   4.9s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10 ..............
[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=10, total=   5.1s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20 ..............
[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20, total=   5.2s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20 ..............
[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20, total=   5.2s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20 ..............
[CV]  ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20, total=   8.1s
[CV] ADASYN__sampling_strategy=0.5, KNN__n_neighbors=20 ..............

[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=20, total=   6.1s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30 .............
[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30, total=   5.9s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30 .............
[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30, total=   6.1s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30 .............
[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30, total=   5.9s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30 .............
[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30, total=   6.2s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30 .............
[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=30, total=   6.1s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=40 .............
[CV]  ADASYN__sampling_strategy=0.75, KNN__n_neighbors=40, total=   6.2s
[CV] ADASYN__sampling_strategy=0.75, KNN__n_neighbors=40 ......

[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=40, total=   6.8s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=40 ................
[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=40, total=   8.7s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=40 ................
[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=40, total=   8.9s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=50 ................
[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=50, total=   7.7s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=50 ................
[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=50, total=   7.1s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=50 ................
[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=50, total=   7.6s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=50 ................
[CV] . ADASYN__sampling_strategy=1, KNN__n_neighbors=50, total=   6.5s
[CV] ADASYN__sampling_strategy=1, KNN__n_neighbors=50 ................
[CV] .

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 17.2min finished


1040.96884 sec


In [12]:
roc_auc_score(y_test, y_validation_preds)

0.8577696393741421

In [15]:
gs_pipeline.best_estimator_

Pipeline(memory=None,
         steps=[('SMOTE',
                 SMOTE(k_neighbors=5, kind='deprecated',
                       m_neighbors='deprecated', n_jobs=1,
                       out_step='deprecated', random_state=101, ratio=None,
                       sampling_strategy=1, svm_estimator='deprecated')),
                ('KNN',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=20, p=2,
                                      weights='uniform'))],
         verbose=False)